Support Vector Classifier 와 Support Vector Regression 에 이용했던 two moon dataset 과 시계열 데이터를 예측하는 nearest neighbors classifiers & regressor 를 학습해 봅니다.

In [1]:
from bokeh.plotting import output_notebook, show
from bokeh.layouts import gridplot
from mydata.visualizer import draw_activate_image
from mydata.visualizer import scatterplot_2class
from soydata.data.classification import make_moons

import numpy as np
np.set_printoptions(precision=5, suppress=True)
import warnings
warnings.filterwarnings('ignore')

output_notebook()

X, labels = make_moons(n_samples=500, xy_ratio=2.0, x_gap=-0.2, y_gap=-0.15, noise=0.1, seed=0)
p = scatterplot_2class(X, labels, height=400, width=400)
show(p)

Loading BokehJS ...

Scikit-learn 에서 제공하는 nearest neighbors 기반 classifier 와 regression 은 init arguments 가 같습니다. 예측 성능에 영향을 주는 값으로는 `n_neighbors`, `weights`, `metric` 이 중요합니다. 예측에 이용하는 최인접이웃의 개수가 작으면 overfitting 이 일어날 가능성이 높으며, 데이터의 개수에 비해 최인접이웃의 개수가 지나치게 많으면 underfitting 이 일어납니다. 

이때 적절한 최인접이웃의 개수는 한 query point q 의 최인접이웃들이 포함된 영역 내의 점들이 서로 비슷한 패턴을 지니는 개수입니다. 밀도가 높은 지역에서는 여러 개의 최인접이웃이 비슷한 값을 지닐 가능성이 높습니다. 하지만 밀도가 낮은 지역에서는 각자가 서로 다른 값을 지닐 가능성이 높습니다. 이러한 점을 해결하기 위하여 예측에 사용하는 최인접이웃들의 영향력이 거리에 반비례하도록 만들 수 있습니다. `weights='uniform'` 은 모든 최인접이웃의 영향력이 같음을 의미하며, `weights='distance'` 는 거리에 반비례하도록 각각의 최인접이웃의 레이블값의 영향력을 부여합니다.

또한 최인접이웃을 탐색할 때 이용하는 거리 척도 `metric` 에 의해서도 그 성능은 달라집니다. 이는 nearest neighbors models 을 이용하는 사용자에 의해 잘 선택되어야 합니다. 최인접이웃 기반 모델들은 이를 이용하기 전에 데이터의 representation 을 의도에 맞도록 변경하는 과정이 필요합니다.

예측 계산 속도에 영향을 가장 많이 주는 요소는 `algorithm` 입니다. 이 값이 `algorithm='brute'` 일 경우, 하나의 query point q 에 대해 모든 reference data 와의 거리를 계산하기 때문에 계산량이 많습니다. `algorithm='ball_tree'` 를 이용하면 ball tree 를 이용하여 근사적으로 최인접이웃을 검색합니다. 조금 정확도가 떨어질 수는 있지만, 속도는 매우 빨라집니다.

앞서 SVM 과 decision tree 의 예시에서 이용했던 activation map 을 그리는 함수와 scatterplot 을 그리는 함수를 이용하여 최인접이웃 모델에 의해 만들어지는 경계면을 확인합니다.

In [2]:
from sklearn.neighbors import KNeighborsClassifier

n_neighbors = 5
weights = 'distance' # ['uniform', 'distance']

model = KNeighborsClassifier(
    n_neighbors=n_neighbors,
    weights=weights, p=2, metric='minkowski',
    algorithm='ball_tree', leaf_size=30
)
model.fit(X, labels)

def prepare_elements(model, X, labels):
    score = model.predict_proba(X)
    score = score[:,1] - score[:,0]
    pred = model.predict(X)    
    accuracy = (pred == labels).sum() / labels.shape[0]
    return score, accuracy

score, accuracy = prepare_elements(model, X, labels)
title = f'{n_neighbors}-NN classifier. accuracy={accuracy:.4}'
p = draw_activate_image(model, X, use_score=True, resolution=100, title=title, height=400, width=400)
p = scatterplot_2class(X, labels, score=score, p=p)
show(p)

거리 척도는 Euclidean distance 로 고정한 뒤, `weights` 와 `n_neighbors` 만 변경하며 클래스 간 경계면의 변화를 살펴봅니다. `weights='distance'` 를 이용할 경우, 여러 개의 최인접이웃을 이용하더라도 멀리 떨어진 점들의 weights 가 죽어들어 경계면 근방에서 아주 큰 변화는 없습니다. 하지만 최인접이웃의 개수가 지나치게 작을 때에는 매우 날카로운 경계면이 학습됩니다. 즉 과적합이 일어나고 있는 것입니다.

또한 최인접이웃의 개수가 어느 정도 많아지면 모든 데이터들과 떨어진 영역 (비어있는 영역)의 점들은 다른 점들과 모두 거리가 멉니다. 그렇기 때문에 그러한 영역의 클래스 별 예측 확률이 비슷할 가능성이 높습니다. 이는 아래 그림의 흰색에 가까운 영역으로 표시됩니다.

In [3]:
grids = []
for weights in ['distance', 'uniform']:
    figures = []
    for n_neighbors in [1, 5, 20, 50]:
        model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
        model.fit(X, labels)
        score, accuracy = prepare_elements(model, X, labels)
        title = f'{n_neighbors}-NN + {weights}. accuracy={accuracy:.4}'
        p = draw_activate_image(model, X, use_score=True, resolution=100, title=title, height=400, width=400)
        p = scatterplot_2class(X, labels, score=score, p=p)
        figures.append(p)
    grids.append(figures)
gp_clf = gridplot(grids)
show(gp_clf)

회귀분석용 데이터를 만듭니다.

In [4]:
from mydata.data import generate_svr_data
from mydata.visualizer.svm import scatterplot_timeseries

x_line, x, y_line, y = generate_svr_data(n_data=200, n_repeats=5)
X_line = x_line.reshape(-1,1)
X = x.reshape(-1,1)

p_tdata = scatterplot_timeseries(x, y, y_line, title='Dataset')
show(p_tdata)

Extraploation 을 확인하기 위하여 reference data x 의 최소, 최대값보다 더 넓은 영역에서 점들을 선택합니다. 회귀모델의 init arguments 도 판별모델과 같기 때문에 이에 대한 설명은 생략합니다. 각각 `n_neighbors` 와 `weights` 를 변화하며 회귀 모형의 패턴을 확인합니다. 최인접이웃의 개수가 많고 weights 가 'uniform' 일수록 underfitting 이 발생하는 것을 확인할 수 있습니다.

In [5]:
from sklearn.neighbors import KNeighborsRegressor

x_test = np.linspace(x.min()-20, x.max()+20, 200).reshape(-1,1)

grids = []
for n_neighbors in [1, 5, 50, 200]:
    figures = []
    for weights in ['distance', 'uniform']:
        model = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights)
        model.fit(x.reshape(-1,1), y)
        y_pred = model.predict(x_test)
        title = f'{n_neighbors}-NN + {weights} regression'
        p = scatterplot_timeseries(x, y, y_line, size=1, height=250, width=600, title=title)
        p = scatterplot_timeseries(x_test.reshape(-1), y_pred, y_pred, size=3, point_color='#d7191c', line_color='#d7191c', p=p)
        figures.append(p)
    grids.append(figures)
gp_reg = gridplot(grids)
show(gp_reg)

Regression 의 경우 최인접이웃들의 y 값의 분산을 이용하여 신뢰구간을 정의할 수 있습니다. 하지만 이 기능은 scikit-learn 에서 제공하지 않으니 직접 구현해 봅니다. scikit-learn 에는 (n, d), (m, d) 크기의 두 개의 행렬의 rows 간의 거리를 계산하는 `pairwise_distances` 함수를 제공합니다. 모든 A 의 rows 에 대해 B 의 rows 간의 `metric` distance 를 측정합니다. 이 값, `dist` 에 argsort(axis=1) 을 수행하면 각 rows 에 대하여 column 의 기준으로 오름차순 정렬이 이뤄집니다. 즉 한 A 의 row 에 대해서 B 의 row 중 거리가 가까운 순서대로 index 가 출력됩니다. 이 값에서 top k 를 선택한 뒤, 이를 column index 인 `cols` 로 flatten 합니다. 각 row 는 k 개씩 반복되니 이에 대한 indices 인 `rows` 도 만듭니다. 그 뒤 distance 에서 rows, cols 의 index 를 가져온 뒤, 다시 (n, k) 형식으로 reshape 을 하면 A 의 모든 row 에 대해서 각각 가장 가까운 B 의 rows 의 인덱스와 거리를 가져올 수 있습니다.

In [6]:
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

np.random.seed(0)

k = 2
A = np.random.random_sample((3, 2))
B = np.random.random_sample((5, 2))
dist = pairwise_distances(A, B, metric='euclidean')

print(f'shape of dist = {dist.shape}\n\ndist')
print(dist, end='\n\n')

indices = dist.argsort(axis=1)
print('Indices of column sorting')
print(indices, end='\n\n')
cols = indices[:,:k].reshape(-1)
rows = np.array([i for i in range(dist.shape[0]) for _ in range(k)], dtype=np.int)

dist_nn = dist[rows, cols].reshape(-1,k)
print('Distance between B and their nearest neighbors')
print(dist_nn)

shape of dist = (3, 5)

dist
[[0.20869 0.53118 0.30612 0.21128 0.78913]
 [0.38421 0.39536 0.18964 0.38229 0.70162]
 [0.24627 0.60041 0.38622 0.31477 0.66073]]

Indices of column sorting
[[0 3 2 1 4]
 [2 3 0 1 4]
 [0 3 2 1 4]]

Distance between B and their nearest neighbors
[[0.20869 0.21128]
 [0.18964 0.38229]
 [0.24627 0.31477]]


Reference data `X_ref`, `y_ref` 와 query vectors 인 `queries`, 그리고 topk `k`, `metric` 을 입력받은 뒤, 위의 과정을 수행합니다. 아래 함수 내 `weight_nn` 은 exp(-dist) 로 모든 rows 에 대한 weights 를 계산한 부분입니다. 이 값을 scikit-learn 의 `normalize` 함수에 입력합니다. 각 row 값의 합을 1 로 정규화합니다. numpy.multiply(A, B) 는 A, B 의 element-wise product 를 계산합니다. weight 와 y 의 값을 곱한 뒤, sum 을 수행하면 weighted average 가 계산됩니다. 이 값을 예측값 `y_pred` 로 이용할 수 있습니다. 그리고 최인접이웃들간의 거리의 stdev 를 계산하여 return 합니다.

In [7]:
def predict_with_std(X_ref, y_ref, queries, k, metric='euclidean'):
    dist = pairwise_distances(queries, X_ref, metric=metric)

    indices = dist.argsort(axis=1)
    cols = indices[:,:k].reshape(-1)
    rows = np.array([i for i in range(dist.shape[0]) for _ in range(k)], dtype=np.int)

    dist_nn = dist[rows, cols].reshape(-1,k)
    weight_nn = np.exp(-dist_nn)
    weight_nn = normalize(weight_nn, norm='l1')

    y_pred = y_ref[cols].reshape(-1, k)
    y_pred = np.multiply(y_pred, weight_nn)
    y_pred = y_pred.sum(axis=1)

    std = dist_nn.std(axis=1)
    return y_pred, std

신뢰구간은 각 xi 에 대하여 (y_pred - 2 x std, y_pred + 2 x std) 로 정의합니다.

In [8]:
X_ref = x.reshape(-1,1)
y_ref = y

y_pred, std = predict_with_std(X_ref, y_ref, x_test, k=20)
y_lower = y_pred - 2 * std
y_upper = y_pred + 2 * std

세 개의 예측선, `y_pred`, `y_lower`, `y_upper` 를 얻었습니다. 이들을 겹쳐 confidence interval 이 포함된 knn regression line 을 그립니다.

In [9]:
title = 'k-NN regression with confidence interval'
p_regvar = scatterplot_timeseries(x, y, y_line, size=1, point_color='grey', line_color='grey', height=400, width=800, title=title)
p_regvar = scatterplot_timeseries(x_test.reshape(-1), y_pred, y_pred, size=3, point_color='#d7191c', line_color='#d7191c', p=p_regvar)
p_regvar = scatterplot_timeseries(x_test.reshape(-1), y_lower, y_lower, size=1, point_color='#abdda4', line_color='#abdda4', p=p_regvar)
p_regvar = scatterplot_timeseries(x_test.reshape(-1), y_upper, y_upper, size=1, point_color='#abdda4', line_color='#abdda4', p=p_regvar)
show(p_regvar)

In [11]:
# from bokeh.io import save

# save(gp_clf, './figures/knn_classifier_various_setting.html')
# save(gp_reg, './figures/knn_regressor_various_setting.html')
# save(p_regvar, './figures/knn_regressor_with_ci.html')